In [ ]:
# jupyter nbconvert --to python test_software_dev_version.ipynb test_software_dev_version.py
%reset -f
import importlib
import numpy as np
import matplotlib.pyplot as plt

import sys

import Frog
import Frog.toolbox as toolbox
import Frog.frog_data_analysis as frog_data_analysis
# importlib.reload(frog_data_analysis)

import pickle
import time
from scipy.optimize import curve_fit


# Load the results

In [ ]:
# The directory where is located the FROG results:
directory = '/home/glebreton/Software/Frog/Doc/Tutorial_files/Optical_analysis_overview'
name_result = 'L_moleculetype_result.p'

GP, L_moleculetype_result = frog_data_analysis.load_result(directory, name_result=name_result, what_to_print=['general info', 'diagram info'])

# Individual Values

## Electrostatic field

In [ ]:
print('The maximal distance used to built the electrostatic environement for the QM calculation (and thus the electric field values stored in electric_field_PE) is ' + str(L_moleculetype_result[0].mtparameter.optparameter.qmparameter.max_pe_distance_neigh) + ' angstrom.')

for sdparameter in L_moleculetype_result[0].mtparameter.dparameter.L_diagram:
    print('This is the diagramparameter of the diagram:', sdparameter.name)
    
    if sdparameter.name == 'electric_field_on_fly':
        print('The maximal distance to built the environment is:' + str(sdparameter.max_distance) + ' angstrom for the electric_field_on_fly diagram.')

print(sdparameter.bin_size[1:3] == (2, 12))


## QM calculations 

In [ ]:
print('There are : ' + str(len(L_moleculetype_result[0].L_molecule)) + ' molecules for this MT.')
n = 150 # the molecule number
my_molecule = L_moleculetype_result[0].L_molecule[n] 
print('Here are the available attribute for this molecule:', my_molecule.L_attribute )

In [ ]:
print('The L_moleculetype_result[0].mtparameter.optparameter.where_to_run_QM is:', L_moleculetype_result[0].mtparameter.optparameter.where_to_run_QM, '. The value [-1] means no selection (all the molecule are treated)') 
print('The L_moleculetype_result[0].mtparameter.optparameter.selection_tool is:', L_moleculetype_result[0].mtparameter.optparameter.selection_tool, '. The value False means no selection (all the molecule are treated)')
print('The size of L_moleculetype_result[0].mtparameter.optparameter.L_QM_todo is:', len(L_moleculetype_result[0].mtparameter.optparameter.L_QM_todo), ' which means that all the molecule are treated since the total number of molecule for this MT is: ', len(L_moleculetype_result[0].L_molecule)) 

In [ ]:
print('my_molecule.IS_QM_run:', my_molecule.IS_QM_run)

In [ ]:
print('Dimension of my_molecule.electric_field_PE', my_molecule.electric_field_PE.shape)


print('Laboratory electric field: EX, EY, EZ', my_molecule.electric_field_PE[0][:3])
print('Laboratory electric field derivative in EX/dX, EY/dX, EZ/dX, EX/dY, EY/dY, EZ/dY, EX/dZ, EY/dZ, EZ/dZ', my_molecule.electric_field_PE[0][3:])
print('Molecular electric field in Ex, Ey, Ez', my_molecule.electric_field_PE[1][:3])
print('Molecular electric field derivative in Ex/dx, Ey/dx, Ez/dx, Ex/dy, Ey/dy, Ez/dy, Ex/dz, Ey/dz, Ez/dz', my_molecule.electric_field_PE[1][3:])


In [ ]:
# Just to check:
print('Laboratory electric field:', my_molecule.electric_field_PE[0][:3])
print('Molecular electric field:', my_molecule.electric_field_PE[1][:3])
rot_matrix =  my_molecule.rot_mat
diff = np.sum((my_molecule.electric_field_PE[1][:3] - toolbox.rotate_1st_order_tensor(rot_matrix.T, my_molecule.electric_field_PE[0][:3]))**2)
print('should be zero:', diff)

print('Laboratory electric field derivative:', my_molecule.electric_field_PE[0][3:])
print('Molecular electric field derivative:', my_molecule.electric_field_PE[1][3:])
rot_matrix =  my_molecule.rot_mat
# here we have to give to  toolbox.rotate_2nd_order_tensor a 3x3 matrix. Therefore, the electric field derivative is converted to this format:
L_dE_lab = np.zeros((3, 3)) # here, L_dE_lab[j][i] = Ei/dj
for j in range(3):
    for i in range(3):
        L_dE_lab[j][i] = my_molecule.electric_field_PE[0][3+j*3+i]

L_dE_mol = toolbox.rotate_2nd_order_tensor(rot_matrix.T, L_dE_lab)
# L_dE_mol is a 3x3 matrix. The convertion to a 1X9 vector to have the same dimension as my_molecule.electric_field_PE[1][3:]
L_dE_mol_listed = [0 for k in range(9)]
for j in range(3):
    for i in range(3):
        L_dE_mol_listed[j*3+i] = L_dE_mol[j][i]

diff = np.sum((my_molecule.electric_field_PE[1][3:] - L_dE_mol_listed)**2)
print('should be zero:', diff)

In [ ]:
diff = np.sum((my_molecule.electric_field_PE-my_molecule.electric_field_on_fly)**2)
print('Should be zero:', diff)

# Diagram

## Electrostatic field:

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'electric_field_PE'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
# type(my_diagram)

In [ ]:
print(my_diagram.value.shape)
print(my_diagram.mean.shape)

In [ ]:
bin_altitude = 0 # 0 if the 'Averaged' geometrical discretization is used

L_xyz = ['x', 'y', 'z']
plt.figure(1, figsize=(10, 7))
plt.title('Laboratory frame', fontsize=25)
for k in range(3):
    plt.plot(my_diagram.axis_observable.value, my_diagram.value[bin_altitude][0][k], label=r'$E_{' + L_xyz[k] + '}$', lw=3)

plt.xlabel(my_diagram.axis_observable.unit.print_unit(), fontsize=25)
plt.ylabel(my_diagram.unit.print_unit(), fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.title('Molecular frame', fontsize=25)
for k in range(3):
    plt.plot(my_diagram.axis_observable.value, my_diagram.value[bin_altitude][1][k], label=r'$E_{' + L_xyz[k] + '}$', lw=3)

plt.xlabel(my_diagram.axis_observable.unit.print_unit(), fontsize=25)
plt.ylabel(my_diagram.unit.print_unit(), fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

In [ ]:
bin_altitude = 0 # 0 if the 'Averaged' geometrical discretization is used
i = 0 # electric_field_direction 0, 1, 2 = x, y, z
L_xyz = ['x', 'y', 'z']
plt.figure(1, figsize=(10, 7))
plt.title('Laboratory frame', fontsize=25)
for j in range(3):
    plt.plot(my_diagram.axis_observable.value,  my_diagram.value[bin_altitude][0][3+j*3+i], label=r'$E_{' + L_xyz[i] + '/d' + L_xyz[j] + '}$', lw=3)

plt.xlabel(my_diagram.axis_observable.unit.print_unit(), fontsize=25)
plt.ylabel(my_diagram.unit.print_unit(), fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.title('Molecular frame', fontsize=25)
for j in range(3):
    plt.plot(my_diagram.axis_observable.value, my_diagram.value[bin_altitude][1][3+j*3+i], label=r'$E_{' + L_xyz[i] + '/d' + L_xyz[j] + '}$', lw=3)

plt.xlabel(my_diagram.axis_observable.unit.print_unit(), fontsize=25)
plt.ylabel(my_diagram.unit.print_unit(), fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

## Polarizability

In [ ]:
myMT = L_moleculetype_result[0]
print('Reference polarizability: \n', myMT.mtparameter.optparameter.L_alpha_ref)

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'alpha'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
# type(my_diagram)

print('Poulation of the polarizability diagram in the molecular frame:', my_diagram.population)
print('my_diagram.mean: \n', my_diagram.mean)
print('my_diagram.sd: \n', my_diagram.sd)


In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'iota'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
# type(my_diagram)

print('Poulation of the polarizability diagram in the laboratory frame:', my_diagram.population)
print('my_diagram.mean: \n', my_diagram.mean)
print('my_diagram.sd: \n', my_diagram.sd)


## First Hyperpolarizability

In [ ]:
for sdparameter in L_moleculetype_result[0].mtparameter.dparameter.L_diagram:
    print('This is the diagramparameter of the diagram:', sdparameter.name)
    
    if sdparameter.analysis_type == 'beta' or sdparameter.analysis_type == 'chi':
        print('The frequency of calculation is: ' + str(sdparameter.frequency) + ' in atomic unit')
        if sdparameter.frequency < 0.001: 
            print('The wave-length of calculation is: +infty in nm')
        else:
            print('The wave-length of calculation is: ' + str(800*0.05686/sdparameter.frequency) + ' in nm')


In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'beta_0_05686'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
# type(my_diagram)

print('Poulation:', my_diagram.population)
print('my_diagram.mean: \n', my_diagram.mean)
print('my_diagram.sd: \n', my_diagram.sd)


In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'chi_0_05686'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
# type(my_diagram)

print('Poulation:', my_diagram.population)
print('my_diagram.mean: \n', my_diagram.mean)
print('my_diagram.sd: \n', my_diagram.sd)
